In [248]:
#####ベイジアン確率的ブロックモデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy import sparse
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

In [249]:
####データの発生####
##データの設定
N = 5000   #ユーザー数
K = 2000   #アイテム数
seg_u = 8   #ユーザーセグメント数
seg_i = 6   #アイテムセグメント数

##セグメントとパラメータを生成
#ユーザーセグメントを生成
alpha01 = np.repeat(100, seg_u)
pi01 = np.random.dirichlet(alpha01, 1).reshape(seg_u)
z01 = np.random.multinomial(1, pi01, N)
z01_vec = np.dot(z01, range(seg_u))
z1_cnt = np.sum(z01, axis=0)
print(np.mean(z01, axis=0))

#アイテムセグメントを生成
alpha02 = np.repeat(100, seg_i)
pi02 = np.random.dirichlet(alpha02, 1).reshape(seg_i)
z02 = np.random.multinomial(1, pi02, K)
z02_vec = np.dot(z02, range(seg_i))
z2_cnt = np.sum(z02, axis=0)
print(np.mean(z02, axis=0))

#観測変数のパラメータの設定
#ユーザーセグメント×アイテムセグメントのベータ事前分布のパラメータを生成
theta0 = np.random.beta(1.0, 6.5, seg_u*seg_i).reshape((seg_u, seg_i))

[0.1306 0.1216 0.138  0.118  0.1244 0.1134 0.1272 0.1268]
[0.148  0.167  0.1825 0.1905 0.1475 0.1645]


In [250]:
##ベルヌーイ分布から観測行列を生成
Data = np.zeros((N, K))
for i in range(seg_u):
    print(i)
    index_z01 = ((z01_vec==i) + 0) * range(1, N+1)
    a = index_z01[index_z01!=0]-1
    
    for j in range(seg_i):
        n = z1_cnt[i] * z2_cnt[j]
        index_z02 = ((z02_vec==j) + 0) * range(1, K+1)
        b = index_z02[index_z02!=0]-1
        y = np.random.binomial(1, theta0[i, j], n).reshape((z1_cnt[i], z2_cnt[j]))
        
        for k in range(z2_cnt[j]):
            Data[z01_vec==i, b[k]] = y[:, k]

#スパース行列に変換
Data_T = Data.T
sparse_data0 = sparse.csc_matrix(1-Data)
sparse_data1 = sparse.csc_matrix(Data)
sparse_data_T0 = sparse.csc_matrix(1-Data_T)
sparse_data_T1 = sparse.csc_matrix(Data_T)

0
1
2
3
4
5
6
7


In [253]:
####マルコフ連鎖モンテカルロ法で確率的ブロックモデルを推定####
##アルゴリズムの設定
R = 2000
burnin = 500
keep = 2
disp = 10
iter = 0
sbeta = 1.5

##事前分布の設定
alpha1 = alpha2 = 1
tau = np.repeat(1, 2)   #ベータ分布の事前分布
alpha1 = np.repeat(1, seg_u)   #ユーザーセグメントのディリクレ事前分布
alpha2 = np.repeat(1, seg_i)   #アイテムセグメントのディリクレ事前分布

##パラメータの真値
z1 = z01_vec
z2 = z02_vec
oldtheta = theta0

##初期値の設定
#混合率の初期値
r1 = np.repeat(1/seg_u, seg_u)
r2 = np.repeat(1/seg_i, seg_i)

#ブロックごとのパラメータの初期値
index_u = np.array(np.floor(np.linspace(0, N, seg_u+1)), dtype='int64')
index_i = np.array(np.floor(np.linspace(0, K, seg_i+1)), dtype='int64')
sortlist1 = np.argsort(np.sum(Data, axis=1))
sortlist2 = np.argsort(np.sum(Data, axis=0))

#パラメータとクラス割当の初期値を設定
oldtheta = np.zeros((seg_u, seg_i))
z1 = np.repeat(0, N)
z2 = np.repeat(0, K)

#ユーザーのクラス割当のインデックスを設定
for i in range(seg_u):
    index1 = sortlist1[index_u[i]:index_u[i+1]]
    z1[index1] = i 

    #アイテムのクラス割当のインデックスを設定
    for j in range(seg_i):
        index2 = sortlist2[index_i[j]:index_i[j+1]]
        z2 [index2] = j

        #セグメントごとのパラメータを決定
        x = Data[index1, :][:, index2]
        oldtheta[i, j] = np.sum(x, dtype='int64') / x.size
        
#混合率の初期値
r1 = np.ones((N, seg_u)) * 1/seg_u
r2 = np.ones((K, seg_i)) * 1/seg_i
        
##パラメータの格納用配列
THETA = np.zeros((seg_u, seg_i, int(R/keep)))
SEG1 = np.zeros((int(R/keep), N))
SEG2 = np.zeros((int(R/keep), K))

##MCMC推定用配列
LLi1 = np.zeros((N, seg_u))
LLi2 = np.zeros((K, seg_i))
index1 = [i for i in range(seg_u)]
index2 = [j for j in range(seg_i)]

In [254]:
keep = 1
rp = 1

In [255]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##ユーザーのセグメント割当を生成
    #パラメータを対数変換
    log_theta1 = np.log(oldtheta)
    log_theta0 = np.log(1-oldtheta)

    #セグメントごとの二項分布の対数尤度を計算
    LLi1 = sparse_data1.dot(log_theta1[:, z2].T) + sparse_data0.dot(log_theta0[:, z2].T)

    #セグメント割当確率のパラメータを設定
    expl = r1 * np.exp(LLi1 - np.max(LLi1, axis=1).reshape((N, 1)))
    z1_rate = expl / np.sum(expl, axis=1).reshape((N, 1))   #セグメント割当確率

    #多項分布からセグメント割当を生成
    Zi1 = np.zeros((N, seg_u))
    for i in range(N):
        Zi1[i, :] = np.random.multinomial(1, z1_rate[i, :], 1)
    z1 = np.array(np.dot(Zi1, range(seg_u)), dtype='int')

    #混合率を更新
    u_sum = np.sum(Zi1, axis=0) + alpha1
    r1 = np.repeat(np.random.dirichlet(u_sum, 1), N, axis=0)   #ディリクレ分布より混合率をサンプリング


    ##アイテムのセグメント割当を生成
    #セグメントごとの二項分布の対数尤度を計算
    LLi2 = sparse_data_T1.dot(log_theta1[z1, :]) + sparse_data_T0.dot(log_theta0[z1, :])

    #セグメント割当確率のパラメータを設定
    expl = r2 * np.exp(LLi2 - np.max(LLi2, axis=1).reshape((K, 1)))
    z2_rate = expl / np.sum(expl, axis=1).reshape((K, 1))   #セグメント割当確率

    #多項分布からセグメント割当を生成
    Zi2 = np.zeros((K, seg_i))
    for j in range(K):
        Zi2[j, :] = np.random.multinomial(1, z2_rate[j, :], 1)
    z2 = np.array(np.dot(Zi2, range(seg_i)), dtype='int')

    #混合率を更新
    i_sum = np.sum(Zi2, axis=0) + alpha2
    r2 = np.repeat(np.random.dirichlet(i_sum, 1), K, axis=0)   #ディリクレ分布より混合率をサンプリング


    ##ベータ分布からパラメータをサンプリング
    #インデックスを作成
    index1 = [(z1==i).nonzero() for i in range(seg_u)]
    index2 = [(z2==j).nonzero() for j in range(seg_i)]

    #セグメント割当ごとにパラメータを生成
    for i in range(seg_u):
        for j in range(seg_i):
            x = Data[index1[i][0], :][:, index2[j][0]]   #割当セグメントに該当するデータを抽出
            y = np.sum(x)
            n = x.size

            #ベータ分布からパラメータをサンプリング
            alpha = tau[0] + y
            beta = tau[1] + n - y
            oldtheta[i, j] = np.random.beta(alpha, beta, 1)


    ##サンプリング結果の保存と確認
    #サンプリング結果の保存
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = oldtheta
        SEG1[mkeep, :] = z1
        SEG2[mkeep, :] = z2

        #パラメータの表示
        if rp%disp==0:
            LL = np.sum(np.sum(LLi1 * Zi1, axis=1))   #対数尤度を計算
            print(rp)
            print(LL)

0
-3502597.1105132354
10
-3325021.0256316587
20
-3325006.8218693854
30
-3325003.4625613615
40
-3325022.5415689386
50
-3325017.2267093062
60
-3325024.195580053
70
-3325015.7327091303
80
-3325010.0056270557
90
-3325008.984832187
100
-3325013.550985382
110
-3325008.1950740824
120
-3325011.339010736
130
-3325012.7063080794
140
-3325012.518137418
150
-3325010.438814717
160
-3325013.7762313094
170
-3325007.2085608877
180
-3325020.3840706777
190
-3325007.8146602046
200
-3325011.9964084434
210
-3325013.0819957773
220
-3325009.205474401
230
-3325024.5743679567
240
-3325017.735083148
250
-3325017.1713882564
260
-3325017.488034498
270
-3325011.8490242716
280
-3325019.161902005
290
-3325012.8831044193
300
-3325016.1459387997
310
-3325007.2187437555
320
-3325014.2079881346
330
-3325017.067629493
340
-3325008.7492874227
350
-3325004.1057072077
360
-3325013.4134676456
370
-3325020.2042037556
380
-3325008.6259356034
390
-3325015.1925513595
400
-3325016.6552403523
410
-3325007.8879120876
420
-3325014.2

IndexError: index 1000 is out of bounds for axis 2 with size 1000